In [45]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [46]:
data = pd.read_csv(r'./data/nursery.csv')

In [47]:
data.count()

parents             12960
has_nurs            12960
form                12960
children            12960
housing             12960
finance             12960
social              12960
health              12960
final evaluation    12960
dtype: int64

There is no missing data in the dataset. So, we can directly move on to the next step.

In [48]:
y =  data['final evaluation']
X = data.drop(labels=['final evaluation'], axis=1)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [69]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=None, impurity='entropy'):
        self.max_depth = max_depth
        self.impurity_name = impurity
        self.decision_tree_pathes = None

        if impurity == 'entropy':
            self.impurity = self.entropy

        elif impurity == 'gini':
            self.impurity = self.gini

    def encode_X(self, X):
        return np.array(X.apply(LabelEncoder().fit_transform))

    def encode_y(self, y):
        self.label_encoder = LabelEncoder()
        return self.label_encoder.fit_transform(y)

    def decode_y(self, y):

        if self.label_encoder is None:
            raise Exception('Label encoder is not initialized')

        return self.label_encdoer.inverse_transform(y)

    def probablity(self, X):
        return np.unique(X,return_counts=True)[1]/X.shape[0]

    def entropy(self, X):
        X = self.probablity(X)

        return np.dot(X, -np.log2(X))

    def gini(self, X):
        X = self.probablity(X)
        return 1 - np.dot(X, X)

    def information_gain(self, X, y):

        if len(X.shape) == 1:
            np.expand_dims(X, axis=1)

        info_gain = list()
        for i in range(X.shape[1]):
            X1 = X[:, i]
            info_gain.append(self.impurity(y)
                             - np.sum(
                                 [self.probablity(X1)[j] *
                                  self.impurity(y[X1 == j])
                                  for j in np.unique(X1)]))

        return np.array(info_gain)

    def split_nodes(self, X, y):

        feature_arg = np.argmax(self.information_gain(X, y))
        X1 = X[:, feature_arg]
        X = np.delete(X, feature_arg, axis=1)

        return [(X[X1 == i], y[X1 == i]) for i in np.unique(X1)]

    def fit(self, X, y):
        X = self.encode_X(X)
        y = self.encode_y(y)

        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=1)

        if self.max_depth is None:
            self.max_depth = X.shape[1]

        Xs_ys = self.split_nodes(X, y)
        for _ in range(self.max_depth-1):
            Xs_ys_new = list()
            for i in range(len(Xs_ys)):
                Xs_ys_new += self.split_nodes(Xs_ys[i][0], Xs_ys[i][1])

            Xs_ys = Xs_ys_new

        return Xs_ys


In [73]:
# model.probablity(X_train_encoded[:,7])

array([0.33275463, 0.33246528, 0.33478009])

In [74]:
model = DecisionTreeClassifier(max_depth=None)
X_train_encoded = model.encode_X(X_train)
y_train_encoded = model.encode_y(y_train)
# model.fit(X_train,y_train)
# model.information_gain(X_train_encoded,y_train_encoded)
# model.probablity(X_train_encoded[:,7])
# model.fit(X_train, y_train)